In [1]:
from future.utils import iteritems
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
import re
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import gensim.downloader

In [3]:
cn_vectors = pd.read_csv("/home/jtvansch/share/embeddings/cn_ce_100_2.csv")

In [4]:
cn_vectors

,token,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,e41,e42,e43,e44,e45,e46,e47,e48,e49,e50
0,a,1.467613,-1.237375,-2.981731,-1.501061,1.877658,1.258956,1.002212,-0.583636,1.460757,...,-3.382263,-1.671850,-1.763873,-1.296559,-0.904948,-4.466372,-1.560971,-1.279335,-2.899061,-1.495396
1,aa,0.308166,-0.688937,-0.152841,0.313727,0.174215,-1.087018,-0.683475,0.485497,-2.590817,...,-1.568199,-1.174421,-1.874193,-1.151193,-4.320134,-1.265643,-6.452040,-3.188478,-1.543746,-4.840497
2,aaa,0.185322,1.004584,0.421218,1.123356,-1.478954,-0.024646,-0.409731,0.797623,-1.180365,...,-2.164749,-1.978481,-2.475305,-2.162855,-2.087740,-2.361140,-2.221941,-2.422956,-2.294493,-2.404330
3,aaahhhhhssssss,0.756566,-1.943488,-0.472578,-0.902454,0.581884,-0.269419,-0.503251,0.711363,-0.168953,...,-1.844517,-1.602908,-1.765803,-1.719397,-1.719696,-1.672052,-7.982901,-1.790639,-1.671304,-4.018158
4,aaannnddd,-0.484201,-1.011751,1.582895,-0.642398,-0.182506,-2.098517,-0.736944,1.230328,-0.776474,...,-2.197775,-2.068716,-2.379195,-2.210611,-2.272923,-2.189353,-2.306276,-2.514430,-2.229361,-2.437102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31289,θbloom,0.131610,1.122219,-2.413387,0.804525,0.579766,1.105433,-1.029753,0.095673,-1.318471,...,-2.153842,-2.073723,-2.334558,-2.180923,-2.272055,-2.330373,-2.292384,-2.558641,-2.194699,-2.444917
31290,θdiff,1.194438,-0.025920,-0.305598,-0.844286,0.213569,0.629932,1.678933,-0.346384,1.660299,...,-3.081275,-1.904265,-1.719283,-1.994834,-3.524750,-1.965578,-1.666395,-3.030027,-1.603763,-2.373290
31291,θturb,2.180069,-1.161924,0.945274,-1.552069,1.037211,0.045158,-1.167034,0.274082,-1.811515,...,-2.734826,-2.252282,-2.191794,-2.336969,-1.921844,-3.080508,-1.903692,-2.214810,-2.111354,-1.810452
31292,аɩѕο,0.357447,1.642604,-0.681744,-0.063346,-0.119409,-0.292735,0.544450,-1.135131,1.047704,...,-2.216517,-1.993241,-2.332905,-2.164743,-2.091177,-2.349440,-2.172904,-2.482840,-2.188882,-2.569129


In [5]:
w2v_model = gensim.downloader.load('word2vec-google-news-300')

In [188]:
train = pd.read_pickle("train.pkl")
test = pd.read_pickle("test.pkl")

In [189]:
train_x = train['token']
train_y = train['label']
train_id = train['sen_id']

In [190]:
test_x = test['token']
test_y = test['label']
test_id = test['sen_id']

In [191]:
vocab = set([item for sublist in train_x for item in sublist] + [item for sublist in test_x for item in sublist])
tag = ['O', 'C', 'E']

In [192]:
vectors =cn_vectors.set_index('token').T.to_dict('list')

<ipython-input-192-9fb74df96829>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  vectors =cn_vectors.set_index('token').T.to_dict('list')


In [193]:
len(vectors['arrows'])

100

In [194]:
zero_300 = np.array([0] * 300)
zero_301 = np.concatenate((np.array([1]), zero_300))

zero_100 = np.array([0] * 100)
zero_101 = np.concatenate((np.array([1]), zero_100))


In [195]:
word2idx = {}

not_present = []

In [196]:
for i in vocab:
    try:
        vec = w2v_model[i]
        vec1 = np.concatenate((np.array([0]), vec))
        word2idx[i]= vec1
        
    except:
        word2idx[i]= zero_301


In [197]:
for i in vocab:
    try:
        vec = vectors[i]
        vec1 = np.concatenate((np.array([0]), vec))
        word2idx[i]= np.concatenate((word2idx[i], vec1))
    except:
        word2idx[i]= np.concatenate((word2idx[i], zero_101))
    

In [198]:
word2idx['dam']

array([ 0.00000000e+00, -5.83496094e-02,  2.92968750e-01,  2.04101562e-01,
       -3.47656250e-01, -4.22363281e-02, -1.76757812e-01,  2.08007812e-01,
       -7.56835938e-03,  2.83203125e-01, -1.85546875e-01,  2.71484375e-01,
       -1.69921875e-01,  4.83398438e-02, -1.74804688e-01, -9.52148438e-02,
        2.62451172e-02, -1.73828125e-01, -1.44531250e-01, -3.66210938e-02,
       -6.25000000e-02,  5.00488281e-02,  5.78613281e-02, -4.63867188e-02,
       -2.05078125e-02,  1.33789062e-01, -2.55859375e-01, -1.36718750e-01,
        1.15234375e-01,  9.17968750e-02, -3.51562500e-01, -2.31445312e-01,
       -6.29882812e-02, -1.81640625e-01,  1.79687500e-01, -1.80664062e-01,
       -2.50000000e-01,  7.47070312e-02,  3.00781250e-01,  1.64062500e-01,
       -2.77343750e-01, -2.51953125e-01,  1.46484375e-01,  2.09960938e-01,
       -1.88476562e-01, -4.49218750e-02, -2.08984375e-01, -1.01318359e-02,
        2.99072266e-02, -1.48437500e-01,  2.73437500e-01,  4.05273438e-02,
       -2.92968750e-01,  

In [199]:
len(vocab)

5177

In [200]:
len(word2idx['guitar'])

402

In [201]:
tag2idx = {t: i for i, t in enumerate(tag)}
idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [202]:
def prepare_sequence(seq, to_idx):
    
    idxs = [to_idx[w] for w in seq]
    idxs = np.array(idxs)
    print(idxs.dtype)
    output_idxs = torch.from_numpy(idxs)
    output_idxs = output_idxs.type(torch.FloatTensor).cuda()
    
    return output_idxs

In [203]:
torch.manual_seed(0)

In [204]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        ''' Initialize the layers of this model.'''
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        #LAYER1 : EMBEDDING LAYER
        # embedding layer that turns words into a vector of a specified size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        #LAYER2 : LSTM
        # the LSTM takes embedded word vectors (of a specified size) as inputs and outputs hidden states of size hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        #LAYER3 : DENSE
        # the linear layer that maps the hidden state output dimension to the number of tags we want as output, tagset_size (in this case this is 3 tags)
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
        
        # initialize the hidden state (see code below)
        self.hidden = self.init_hidden()

        
    def init_hidden(self):

        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        
        #return(torch.randn(2, 1, self.hidden_dim).cuda(),
        #    torch.randn(2, 1, self.hidden_dim).cuda())
        return (torch.zeros(2, 1, self.hidden_dim).cuda(),
                torch.zeros(2, 1, self.hidden_dim).cuda())
    
    def forward(self, sentence):
        
        embeds = sentence
        
        lstm_out, self.hidden = self.lstm((embeds.view(len(sentence), 1, -1)), self.hidden)
        #print(lstm_out.shape)
        
        
        # get the scores for the most likely tag for a word
        tag_scores = self.hidden2tag(lstm_out.view(len(sentence), -1))
        
        return tag_scores

In [205]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
torch.cuda.set_device(3)
print(torch.cuda.current_device())

True
3
3


In [206]:
# the embedding dimension defines the size of our word vectors
# for our simple vocabulary and training set, we will keep these small
EMBEDDING_DIM = 402
HIDDEN_DIM = 120

# instantiate our model
model1 = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))
model1.cuda()

# define our loss and optimizer
loss_function =  nn.CrossEntropyLoss()  #nn.NLLLoss()   
optimizer = optim.SGD(model1.parameters(), lr=0.1)       ##SGD


In [207]:
train_x

0      [i, used, to, get, terrible, headaches, from, ...
1      [peters, s, pain, and, symptoms, were, caused,...
2      [in, the, years, since, the, adm, scandal, of,...
3      [a, first, revolution, was, triggered, by, the...
4      [when, they, took, the, floor, against, a, tea...
                             ...                        
797    [using, solar, electricity, instead, of, conve...
798    [fatigue, corrosion, and, stress, corrosion, a...
799    [the, electromagnetic, em, radiation, from, th...
800    [it, gets, a, little, bit, chicken, first, or,...
801    [the, microphone, converts, sound, into, an, e...
Name: token, Length: 802, dtype: object

In [208]:

test_x

0      [many, adults, retain, scars, from, acne, brea...
1      [we, estimate, a, wind, speed, associated, wit...
2      [outbreaks, caused, by, the, oral, vaccine, s,...
3      [production, and, investigation, of, such, a, ...
4      [the, drugs, he, sold, had, caused, the, overd...
                             ...                        
196    [the, real, possibility, of, total, engulfment...
197    [we, find, evidence, that, ernst, suffered, a,...
198    [the, influx, caused, a, further, drain, on, t...
199    [ngu, nongonococcal, urethritis, is, an, infec...
200    [eye, discomfort, from, this, staring, effect,...
Name: token, Length: 201, dtype: object

In [209]:
training_input_data = []
for i in range(len(train_x)):
    sen = train_x[i]
    tag = train_y[i]
    sen_t = prepare_sequence(sen, word2idx)
    tag_t = prepare_sequence(tag, tag2idx)
    training_input_data.append((sen_t, tag_t))
    
    
testing_input_data = []
for i in range(len(test_x)):
    sen = test_x[i]
    tag = test_y[i]
    sen_t = prepare_sequence(sen, word2idx)
    tag_t = prepare_sequence(tag, tag2idx)
    testing_input_data.append((sen_t, tag_t))
    
    

float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float6

float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float64
int64
float6

In [210]:
len(train_x[0])

20

In [211]:
len(training_input_data[0][0])

20

In [212]:
n_epochs = 15
score_save = []

start_time = time.time()
for epoch in range(n_epochs):
    
    epoch_loss = 0.0
    count=0
    
    for sentence, tags in training_input_data:
        if len(sentence)>0:
            count+=1
            
            model1.zero_grad()

            # zero the hidden state of the LSTM, this detaches it from its history
            model1.hidden = model1.init_hidden()

            
            # forward 
           
            tags = tags.type(torch.LongTensor).cuda()
            
            tag_scores = model1(sentence)
            
            # Loss
            tag_scores = tag_scores.type(torch.FloatTensor).cuda()
            #targets = targets.type(torch.LongTensor).cuda()
            if epoch==n_epochs-1:
                score_save.append(tag_scores.tolist())
           
            loss = loss_function(tag_scores, tags)
            #print(loss)
            epoch_loss += loss.item()
            loss.backward()

            # update the model parameters with optimizer.step()
            optimizer.step()
        
    
    print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(train_x)))
    print("--- %s seconds ---" % (time.time() - start_time))

print("Total time taken : ")
print("--- %s seconds ---" % (time.time() - start_time))

Epoch: 1, loss: 0.36747
--- 4.530977487564087 seconds ---
Epoch: 2, loss: 0.29355
--- 9.030852556228638 seconds ---
Epoch: 3, loss: 0.25651
--- 13.525933027267456 seconds ---
Epoch: 4, loss: 0.22290
--- 18.027941465377808 seconds ---
Epoch: 5, loss: 0.18905
--- 22.530105352401733 seconds ---
Epoch: 6, loss: 0.15922
--- 27.038546085357666 seconds ---
Epoch: 7, loss: 0.13375
--- 31.540041208267212 seconds ---
Epoch: 8, loss: 0.11448
--- 36.03359866142273 seconds ---
Epoch: 9, loss: 0.09530
--- 40.528313875198364 seconds ---
Epoch: 10, loss: 0.08627
--- 45.02537822723389 seconds ---
Epoch: 11, loss: 0.07296
--- 49.520262241363525 seconds ---
Epoch: 12, loss: 0.06564
--- 54.01691770553589 seconds ---
Epoch: 13, loss: 0.05167
--- 58.51698303222656 seconds ---
Epoch: 14, loss: 0.04991
--- 63.02384352684021 seconds ---
Epoch: 15, loss: 0.04212
--- 67.57584190368652 seconds ---
Total time taken : 
--- 67.57665228843689 seconds ---


In [213]:
#Evaluation for training data

pred_tag = []
actual_tag = []
for sen, tag in training_input_data:
    tag_scores = model1(sen)
    _, predicted_tags = torch.max(tag_scores, 1)
    pred = predicted_tags.cpu().tolist()
    pred = [idx2tag[idx] for idx in pred]
    pred_tag.append(pred)
    tag = tag.cpu().tolist()
    act = [idx2tag[idx] for idx in tag]
    actual_tag.append(act)


In [214]:
pred_tag_f = [item for sublist in pred_tag for item in sublist]
actual_tag_f = [item for sublist in actual_tag for item in sublist]

In [215]:
from sklearn.metrics import classification_report

label_names = ['O', 'C', 'E']

print(classification_report(actual_tag_f, pred_tag_f,target_names=label_names))

              precision    recall  f1-score   support

           O       0.88      0.70      0.78       890
           C       0.96      0.65      0.77       887
           E       0.96      0.99      0.98     13185

    accuracy                           0.96     14962
   macro avg       0.93      0.78      0.84     14962
weighted avg       0.96      0.96      0.95     14962



In [216]:
#Evaluation for testing data

pred_tag = []
actual_tag = []
for sen, tag in testing_input_data:
    tag_scores = model1(sen)
    _, predicted_tags = torch.max(tag_scores, 1)
    pred = predicted_tags.cpu().tolist()
    pred = [idx2tag[idx] for idx in pred]
    pred_tag.append(pred)
    tag = tag.cpu().tolist()
    act = [idx2tag[idx] for idx in tag]
    actual_tag.append(act)


In [217]:
pred_tag_f = [item for sublist in pred_tag for item in sublist]
actual_tag_f = [item for sublist in actual_tag for item in sublist]

In [218]:
from sklearn.metrics import classification_report

label_names = ['O', 'C', 'E']

print(classification_report(actual_tag_f, pred_tag_f,target_names=label_names))

              precision    recall  f1-score   support

           O       0.53      0.42      0.47       224
           C       0.74      0.41      0.53       217
           E       0.94      0.97      0.95      3516

    accuracy                           0.91      3957
   macro avg       0.74      0.60      0.65      3957
weighted avg       0.90      0.91      0.90      3957



In [ ]:
def clean_x(sen):
    sen = re.sub('[^a-zA-Z0-9 \n\.]', ' ', sen)
    sen = sen.replace('.', ' ')
    sen = sen.lower()
    sen = sen.split()
    return sen

def clean_y(rel):
    cause = rel[0]
    cause = re.sub('[^a-zA-Z0-9 \n\.]', ' ', cause)
    cause = cause.lower().split()
    effect = rel[1]
    effect = re.sub('[^a-zA-Z0-9 \n\.]', ' ', effect)
    effect = effect.lower().split()

    return ((cause, effect))

In [ ]:
#TEST WITH ANY INPUT SENTENCE


def test_with_sen(input_sen):
    input_sen = clean_x(input_sen)
    
    inputs = prepare_sequence(input_sen, word2idx)

    tag_scores = model1(inputs)


    _, predicted_tags = torch.max(tag_scores, 1)

    print("Input sentence: ", input_sen)
    for i in range(0,len(input_sen)):
        t = ''
        if predicted_tags[i].item()==0:
            t = 'Other'
        elif predicted_tags[i].item()==1:
            t = 'Cause'
        elif predicted_tags[i].item()==2:
            t = 'Effect'
        #elif predicted_tags[i].item==3:
        #    t = 'B-Effect'
        #else:
        #    t = 'I-Effect'
        print(predicted_tags[i].item())
        print(input_sen[i] , ': ', t )
    return
    

In [ ]:
test_sen = 'Smoking causes cancer'
test_with_sen(test_sen)

In [ ]:
a = ' '.join(test_x[46])

print(a)
test_with_sen(a)